In [12]:
import pandas as pd
import geopandas as gp
import numpy as np
import os
import math

In [155]:
# DONT RUN
for filename in os.listdir('../raw/slope-aspect/'):
    if filename.endswith('.zip'):
        zip_f = zipfile.ZipFile("../raw/slope-aspect/"+filename, "r")
        f = filename.strip("NASADEM_SC_").strip(".zip")
#         zip_f.extract("../raw/slope-aspect/hgt/"+f+".hgt")
        zip_f.extract(f+".aspect")
        zip_f.close()
        print(filename)

FileNotFoundError: [Errno 2] No such file or directory: '../raw/slope-aspect/'

In [6]:
file_path = '../raw/slope-aspect/aspect-raw/'

In [9]:
# for filename in os.listdir(file_path):
#     filename_noext = filename.strip('.hg.hgt')
#     os.rename(file_path+filename, file_path+filename_noext+'.hgt')

In [ ]:
final = pd.DataFrame()
for filename in os.listdir(file_path):
    lons = []
    lats = []
    lon_tile = int(filename[4:7])
    lat_tile = int(filename[1:3])
    
    siz = os.path.getsize(file_path + filename)
    dim = int(math.sqrt(siz/2))

    assert dim*dim*2 == siz, 'Invalid file size'

    data = np.fromfile(file_path + filename, np.dtype('>i2'), dim*dim).reshape((dim, dim))

    for i in range(0, 3601):
        lon = lon_tile + 1/7200 + i*(1/3601)
        lat = lat_tile + 1/7200 + i*(1/3601)
        lons.append(lon)
        lats.append(lat)
        
    tile_df = pd.DataFrame(data)
    tile_df.index = lats
    tile_df.T
    tile_df.index = lons
    tile_df.T
    tile_final = tile_df.stack().reset_index()
    tile_final = tile_final.rename(columns = {'level_0': 'lon', 'level_1': 'lat', 0: 'aspect'})
    
    final = final.append(tile_final)